# urls.py
- Django项目的URL声明，就像你网站的目录

## Django如何处理请求
- 当用户从Django支持的站点请求页面时，会进行下面的步骤：
    - Django确定要使用的根URLconf模块
    - 加载Python模块并查找变量urlpatterns，应该为django.urls.path或django.urls.re_path实例
    - Django按顺序遍历每个URL模式，并在匹配请求的URL的第一个模式停止
    - 一旦其中一个URL模式匹配，Django会导入并调用给定的视图
    - 如果没有一个匹配，则会引发异常

## 方法函数

In [ ]:
'''
path(route,view,kwargs,name)
repath(route,view,kwargs,name)
route：
    1.参数应该是一个字符串或gettext_lazy()，可能包含尖括号，用于捕获url的一部分并将其作为关键字参数发送到视图
    2.参数应该是一个字符串或gettext_lazy()，其中包含与python的兼容的正则表达式re模块，字符串通常使用原始字符串语法(r''),\d而无需使用零一个反斜杠转义反斜杠
view：一个视图函数或as_view()基于类的视图，也可以是一个django.urls.include()
kwargs：允许您将其他参数传递给视图函数或方法
'''
from django.urls import include, path

urlpatterns = [
    path('index/', views.index, name='main-view'),
    path('bio/<username>/', views.bio, name='bio'),
    path('articles/<slug:title>/', views.article, name='article-detail'),
    path('weblog/', include('blog.urls')),
    re_path(r'^index/$', views.index, name='index'),
    re_path(r'^bio/(?P<username>\w+)/$', views.bio, name='bio'),
]


'''
include(module,namespace=None)
include(pattern_list)
include((pattern_list,app_namespace),namespace=None)
    module：URLconf模块
    namespace(str)：包含url条目的实例名称空间
    pattern_list：path()或re_path()实例
    app_namespace(str)：包含的URL条目的应用程序命名空间
'''
urlpatterns = [
    path('weblog/', include('blog.urls'))
]

'''
register_converter()：自定义转换器
'''

'''
static(prefix,view=django.views.static.serve,**kwargs)：函数在调试模式下返回服务文件的URL模式
'''


## 路径转换器
### 转换器类型
- str- 匹配除路径分隔符之外的任何非空字符串'/'。如果转换器未包含在表达式中，则这是默认值。
- int - 匹配零或任何正整数。返回一个int。
- slug - 匹配由ASCII字母或数字组成的任何slug字符串，以及连字符和下划线字符。例如， building-your-1st-django-site。
- uuid - 匹配格式化的UUID。要防止多个URL映射到同一页面，必须包含短划线，并且字母必须为小写。例如，075194d3-6885-417e-a8a8-6c931e272f00。返回一个 UUID实例。
- path- 匹配任何非空字符串，包括路径分隔符 '/'。这允许您匹配完整的URL路径，而不仅仅是URL路径的一部分str。

In [ ]:
# 使用自带的路径转换器
from django.urls import path

from . import views

urlpatterns = [
    path('articles/2003/', views.special_case_2003),
    path('articles/<int:year>/', views.year_archive),
    path('articles/<int:year>/<int:month>/', views.month_archive),
    path('articles/<int:year>/<int:month>/<slug:slug>/', views.article_detail),
]

In [ ]:
# 注册自定义路径转换器
'''
包含以下内容的类：
    一个regexclass属性，作为字符串
    - to_python(self,value)方法：处理匹配的字符串转换成要传递到视图函数的类型
    - to_url(self,vlaue)方法：用于处理将python类型转换为要在URL中使用的字符串
'''
class FourDigitYearConverter:
    regex = '[0-9]{4}'

    def to_python(self, value):
        return int(value)

    def to_url(self, value):
        return '%04d' % value
    
# 在url.py中使用注册的自定义路径转换器
from django.urls import path, register_converter

from . import converters, views

register_converter(converters.FourDigitYearConverter, 'yyyy')

urlpatterns = [
    path('articles/2003/', views.special_case_2003),
    path('articles/<yyyy:year>/', views.year_archive),
    ...
]

## 使用正则表达式
- 使用re_path()而不是path()
- 命名方式
    - 命名正则表达式组：(?P\< name \>pattern)：name-组的名称，pattern是要匹配的模式
    - 未命名正则表达式组：(pattern)

In [ ]:
from django.urls import path, re_path

from . import views

urlpatterns = [
    path('articles/2003/', views.special_case_2003),
    re_path(r'^articles/(?P<year>[0-9]{4})/$', views.year_archive),
    repath(r'^articles/([0-9]{4})/$',views.year_archive)
    re_path(r'^articles/(?P<year>[0-9]{4})/(?P<month>[0-9]{2})/$', views.month_archive),
    re_path(r'^articles/(?P<year>[0-9]{4})/(?P<month>[0-9]{2})/(?P<slug>[\w-]+)/$', views.article_detail),
]

## 错误处理

In [ ]:
# 404
'''
如果要使用自己的html文件，需要在模板树的顶层创建一个404.html模板
'''
from django.http import Http404
from django.shortcuts import render
from polls.models import Poll

def detail(request, poll_id):
    try:
        p = Poll.objects.get(pk=poll_id)
    except Poll.DoesNotExist:
        raise Http404("Poll does not exist")
    return render(request, 'polls/detail.html', {'poll': p})

In [ ]:
# 自定义错误视图
# 在URLconf中指定处理错误的视图函数
handler404 = 'mysite.views.my_custom_page_not_found_view'  # 处理404
handler500 = 'mysite.views.my_custom_error_view'
handler403 = 'mysite.views.my_custom_permission_denied_view'
handler400 = 'mysite.views.my_custom_bad_request_view'

## 实例

In [ ]:
# 主url.py文件
from django.contrib import admin
from django.urls import path,include
from django.conf import settings
from django.conf.urls.static import static

urlpatterns = [
    path('admin/', admin.site.urls),
    # 注：这个namespace是include中的！
    path('umeng/',include('umeng.urls',namespace='umeng')),
] + static(settings.MEDIA_URL,document_root=settings.MEDIA_ROOT) # media

In [ ]:
# umeng/url.pt文件
from django.urls import path,re_path
from umeng.views import * # 用于导入对应链接的处理函数

app_name = 'umeng' # 如果在主url.py中用到了namespace，那么就需要在这里定义，否则会报错

urlpatterns = [
    re_path(r'^(\w+)/$',Index,name='appdata') # url的传参模式
]

# umeng中的html文件，其中参数可以是数字，字符串，变量，字符串必须有引号
<li><a href="{% url 'umeng:appdata' 'android' %}">安卓</a></li>

# umeng中的view.py文件
def Index(request,appType):
    print(appType)# appType为传入的参数，即android